In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import pickle

from bs4 import BeautifulSoup
from census import Census
from us import states

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

Census API

https://api.census.gov/data/2017/zbp <br>
https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf <br>
https://github.com/datamade/census

Data sources

"MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS)"
https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_17_5YR_S1903&prodType=table 
<br>
https://simplemaps.com/data/us-zips

Census interactive tables: <br>
https://data.census.gov/cedsci/table?hidePreview=true&tid=ACSST1Y2018.S1901&t=Income%20%28Households,%20Families,%20Individuals%29&g=0400000US01&vintage=2018

What is ZCTA? <br>
https://www.census.gov/programs-surveys/geography/guidance/geo-areas/zctas.html

In [ ]:
data = pd.read_csv('../data/csv_name.csv')

point in polygon algorithm

In [2]:
census_api_key = "inser your own api key here.a"

In [5]:
pd.read_csv('./data/income_data.csv')

,Unnamed: 0,ZCTA,n_households,med_income,population,density,city,state_id,state_name,lat,lng
0,0,601,5818,11757,17242,111.4,Adjuntas,PR,Puerto Rico,18.18004,-66.75218
1,1,602,12719,16190,38442,523.5,Aguada,PR,Puerto Rico,18.36073,-67.17517
2,2,603,19009,16645,48814,667.9,Aguadilla,PR,Puerto Rico,18.45439,-67.12202
3,3,606,1959,13387,6437,60.4,Maricao,PR,Puerto Rico,18.16724,-66.93828
4,4,610,9120,18741,27073,312.0,Anasco,PR,Puerto Rico,18.29032,-67.12243
...,...,...,...,...,...,...,...,...,...,...,...
33094,33094,99923,0,31250,15,2.1,Hyder,AK,Alaska,55.97796,-130.03671
33095,33095,99925,327,48646,927,5.7,Klawock,AK,Alaska,55.55767,-132.97627
33096,33096,99926,485,57969,1635,4.2,Metlakatla,AK,Alaska,55.12617,-131.48928
33097,33097,99927,41,17981,38,0.2,Point Baker,AK,Alaska,56.25100,-133.37571


### Reading

In [6]:
inc = pd.read_csv('./data/ACS_17_5YR_S1903/ACS_17_5YR_S1903_with_ann.csv')
state = pd.read_csv('./data/simplemaps_uszips_basicv1.7/uszips.csv')

In [4]:
pd.set_option('display.max_columns',250)
inc.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC02,HC01_MOE_VC02,HC02_EST_VC02,HC02_MOE_VC02,HC03_EST_VC02,HC03_MOE_VC02,HC01_EST_VC04,HC01_MOE_VC04,HC02_EST_VC04,HC02_MOE_VC04,HC03_EST_VC04,HC03_MOE_VC04,HC01_EST_VC05,HC01_MOE_VC05,HC02_EST_VC05,HC02_MOE_VC05,HC03_EST_VC05,HC03_MOE_VC05,HC01_EST_VC06,HC01_MOE_VC06,HC02_EST_VC06,HC02_MOE_VC06,HC03_EST_VC06,HC03_MOE_VC06,HC01_EST_VC07,HC01_MOE_VC07,HC02_EST_VC07,HC02_MOE_VC07,HC03_EST_VC07,HC03_MOE_VC07,HC01_EST_VC08,HC01_MOE_VC08,HC02_EST_VC08,HC02_MOE_VC08,HC03_EST_VC08,HC03_MOE_VC08,HC01_EST_VC09,HC01_MOE_VC09,HC02_EST_VC09,HC02_MOE_VC09,HC03_EST_VC09,HC03_MOE_VC09,HC01_EST_VC10,HC01_MOE_VC10,HC02_EST_VC10,HC02_MOE_VC10,HC03_EST_VC10,HC03_MOE_VC10,HC01_EST_VC12,HC01_MOE_VC12,HC02_EST_VC12,HC02_MOE_VC12,HC03_EST_VC12,HC03_MOE_VC12,HC01_EST_VC13,HC01_MOE_VC13,HC02_EST_VC13,HC02_MOE_VC13,HC03_EST_VC13,HC03_MOE_VC13,HC01_EST_VC16,HC01_MOE_VC16,HC02_EST_VC16,HC02_MOE_VC16,HC03_EST_VC16,HC03_MOE_VC16,HC01_EST_VC17,HC01_MOE_VC17,HC02_EST_VC17,HC02_MOE_VC17,HC03_EST_VC17,HC03_MOE_VC17,HC01_EST_VC18,HC01_MOE_VC18,HC02_EST_VC18,HC02_MOE_VC18,HC03_EST_VC18,HC03_MOE_VC18,HC01_EST_VC19,HC01_MOE_VC19,HC02_EST_VC19,HC02_MOE_VC19,HC03_EST_VC19,HC03_MOE_VC19,HC01_EST_VC22,HC01_MOE_VC22,HC02_EST_VC22,HC02_MOE_VC22,HC03_EST_VC22,HC03_MOE_VC22,HC01_EST_VC23,HC01_MOE_VC23,HC02_EST_VC23,HC02_MOE_VC23,HC03_EST_VC23,HC03_MOE_VC23,HC01_EST_VC24,HC01_MOE_VC24,HC02_EST_VC24,HC02_MOE_VC24,HC03_EST_VC24,HC03_MOE_VC24,HC01_EST_VC25,HC01_MOE_VC25,HC02_EST_VC25,HC02_MOE_VC25,HC03_EST_VC25,HC03_MOE_VC25,HC01_EST_VC26,HC01_MOE_VC26,HC02_EST_VC26,HC02_MOE_VC26,HC03_EST_VC26,HC03_MOE_VC26,HC01_EST_VC27,HC01_MOE_VC27,HC02_EST_VC27,HC02_MOE_VC27,HC03_EST_VC27,HC03_MOE_VC27,HC01_EST_VC28,HC01_MOE_VC28,HC02_EST_VC28,HC02_MOE_VC28,HC03_EST_VC28,HC03_MOE_VC28,HC01_EST_VC29,HC01_MOE_VC29,HC02_EST_VC29,HC02_MOE_VC29,HC03_EST_VC29,HC03_MOE_VC29,HC01_EST_VC30,HC01_MOE_VC30,HC02_EST_VC30,HC02_MOE_VC30,HC03_EST_VC30,HC03_MOE_VC30,HC01_EST_VC33,HC01_MOE_VC33,HC02_EST_VC33,HC02_MOE_VC33,HC03_EST_VC33,HC03_MOE_VC33,HC01_EST_VC34,HC01_MOE_VC34,HC02_EST_VC34,HC02_MOE_VC34,HC03_EST_VC34,HC03_MOE_VC34,HC01_EST_VC35,HC01_MOE_VC35,HC02_EST_VC35,HC02_MOE_VC35,HC03_EST_VC35,HC03_MOE_VC35,HC01_EST_VC36,HC01_MOE_VC36,HC02_EST_VC36,HC02_MOE_VC36,HC03_EST_VC36,HC03_MOE_VC36,HC01_EST_VC37,HC01_MOE_VC37,HC02_EST_VC37,HC02_MOE_VC37,HC03_EST_VC37,HC03_MOE_VC37,HC01_EST_VC38,HC01_MOE_VC38,HC02_EST_VC38,HC02_MOE_VC38,HC03_EST_VC38,HC03_MOE_VC38,HC01_EST_VC41,HC01_MOE_VC41,HC02_EST_VC41,HC02_MOE_VC41,HC03_EST_VC41,HC03_MOE_VC41,HC01_EST_VC42,HC01_MOE_VC42,HC02_EST_VC42,HC02_MOE_VC42,HC03_EST_VC42,HC03_MOE_VC42,HC01_EST_VC43,HC01_MOE_VC43,HC02_EST_VC43,HC02_MOE_VC43,HC03_EST_VC43,HC03_MOE_VC43,HC01_EST_VC44,HC01_MOE_VC44,HC02_EST_VC44,HC02_MOE_VC44,HC03_EST_VC44,HC03_MOE_VC44,HC01_EST_VC47,HC01_MOE_VC47,HC02_EST_VC47,HC02_MOE_VC47,HC03_EST_VC47,HC03_MOE_VC47,HC01_EST_VC48,HC01_MOE_VC48,HC02_EST_VC48,HC02_MOE_VC48,HC03_EST_VC48,HC03_MOE_VC48,HC01_EST_VC49,HC01_MOE_VC49,HC02_EST_VC49,HC02_MOE_VC49,HC03_EST_VC49,HC03_MOE_VC49,HC01_EST_VC50,HC01_MOE_VC50,HC02_EST_VC50,HC02_MOE_VC50,HC03_EST_VC50,HC03_MOE_VC50,HC01_EST_VC51,HC01_MOE_VC51,HC02_EST_VC51,HC02_MOE_VC51,HC03_EST_VC51,HC03_MOE_VC51,HC01_EST_VC52,HC01_MOE_VC52,HC02_EST_VC52,HC02_MOE_VC52,HC03_EST_VC52,HC03_MOE_VC52,HC01_EST_VC53,HC01_MOE_VC53,HC02_EST_VC53,HC02_MOE_VC53,HC03_EST_VC53,HC03_MOE_VC53
0,Id,Id2,Geography,Number; Estimate; Households,Number; Margin of Error; Households,Percent Distribution; Estimate; Households,Percent Distribution; Margin of Error; Households,Median income (dollars); Estimate; Households,Median income (dollars); Margin of Error; Hous...,Number; Estimate; Households - One race-- - White,Number; Margin of Error; Households - One race...,Percent Distribution; Estimate; Households - O...,Percent Distribution; Margin of Error; Househo...,Median income (dollars); Estimate; Households ...,Median income (dollars); Margin of Error; Hous...,Number; Estimate; Households - On

In [5]:
inc = inc[['GEO.id2', 'HC01_EST_VC02', 'HC03_EST_VC02']]
inc.columns = ['ZCTA', 'n_households', 'med_income']

In [6]:
inc.drop([0], inplace = True)
inc.reset_index(drop = True, inplace = True)
inc.head()

,ZCTA,n_households,med_income
0,00601,5818,11757
1,00602,12719,16190
2,00603,19009,16645
3,00606,1959,13387
4,00610,9120,18741


### Clean income

In [7]:
inc.dtypes

ZCTA            object
n_households    object
med_income      object
dtype: object

In [8]:
inc['med_income'].value_counts()

-        2164
51250      76
46250      72
48750      72
41250      66
         ... 
58780       1
36676       1
67737       1
59556       1
48190       1
Name: med_income, Length: 19651, dtype: int64

In [9]:
# Clean commas
inc['med_income'] = inc['med_income'].apply(lambda x: x.replace(',', ''))

In [10]:
# Clean 2500-. These will be treated as 2500
def clean_2500(income):
    if (income == '2500-'):
        return '2500'
    else:
        return income
    
inc['med_income'] = inc['med_income'].apply(clean_2500)

In [11]:
# Clean 250000+. These will be treated as 250000
inc['med_income'] = inc['med_income'].apply(lambda x: x.replace('+', ''))

In [12]:
# Interpolate hyphens: hyphens will adopt the value of the nearest numeric zip
for i in range(len(inc['med_income'])):
    if (inc.loc[i, 'med_income'] == '-'):
        numeric_neighbor = False
        neighbor_dist = 0
        while not numeric_neighbor:
            neighbor_dist += 1
            neighbor = inc.loc[i - neighbor_dist, 'med_income']
            if (neighbor != '-'):
                numeric_neighbor = True
               
        inc.loc[i, 'med_income'] = neighbor

In [13]:
# Convert types
inc['n_households'] = pd.to_numeric(inc['n_households'])
inc['med_income'] = pd.to_numeric(inc['med_income'])
inc['ZCTA'] = pd.to_numeric(inc['ZCTA'])

In [14]:
inc.dtypes

ZCTA            int64
n_households    int64
med_income      int64
dtype: object

In [15]:
inc[inc['ZCTA'] == 2110]

,ZCTA,n_households,med_income
464,2110,1347,108438


In [16]:
inc['med_income'].describe()

count     33120.000000
mean      56683.047313
std       24388.928271
min        2500.000000
25%       41250.000000
50%       51802.000000
75%       65824.250000
max      250000.000000
Name: med_income, dtype: float64

In [17]:
inc.isnull().sum()

ZCTA            0
n_households    0
med_income      0
dtype: int64

### Merge state, coordinates data

In [18]:
state = state[['zip', 'population', 'density', 'city', 'state_id', 'state_name', 'lat', 'lng']]

In [19]:
state.isnull().sum()

zip           0
population    0
density       0
city          0
state_id      0
state_name    0
lat           0
lng           0
dtype: int64

In [20]:
df = pd.merge(inc, state, left_on = 'ZCTA', right_on = 'zip')

In [21]:
df.isnull().sum()

ZCTA            0
n_households    0
med_income      0
zip             0
population      0
density         0
city            0
state_id        0
state_name      0
lat             0
lng             0
dtype: int64

In [22]:
df[df['ZCTA'] != df['zip']] # making sure all zips match

,ZCTA,n_households,med_income,zip,population,density,city,state_id,state_name,lat,lng


In [23]:
df.drop(columns = 'zip', inplace = True)

In [24]:
df.dtypes

ZCTA              int64
n_households      int64
med_income        int64
population        int64
density         float64
city             object
state_id         object
state_name       object
lat             float64
lng             float64
dtype: object

In [25]:
df.head()

,ZCTA,n_households,med_income,population,density,city,state_id,state_name,lat,lng
0,601,5818,11757,17242,111.4,Adjuntas,PR,Puerto Rico,18.18004,-66.75218
1,602,12719,16190,38442,523.5,Aguada,PR,Puerto Rico,18.36073,-67.17517
2,603,19009,16645,48814,667.9,Aguadilla,PR,Puerto Rico,18.45439,-67.12202
3,606,1959,13387,6437,60.4,Maricao,PR,Puerto Rico,18.16724,-66.93828
4,610,9120,18741,27073,312.0,Anasco,PR,Puerto Rico,18.29032,-67.12243


In [26]:
df[df['ZCTA'] == 2110]

,ZCTA,n_households,med_income,population,density,city,state_id,state_name,lat,lng
464,2110,1347,108438,2449,3693.1,Boston,MA,Massachusetts,42.35831,-71.05175


In [27]:
df.groupby('state_id')['med_income'].mean().sort_values(ascending = False)

state_id
NJ    91124.322689
CT    86059.567376
MD    85136.854701
MA    81799.586592
RI    74140.675325
NH    72991.883065
NY    69123.387960
CA    68681.324248
DC    68548.019231
WY    65459.814607
HI    65000.829787
ND    64237.375979
DE    64230.761194
VA    62108.263393
CO    61430.849524
WA    61395.394295
UT    61182.832753
NV    60953.428571
MN    60671.119910
IL    59809.668836
VT    58598.239216
WI    57268.873057
IA    56943.158458
PA    56908.195543
NE    56744.194492
TX    55850.844444
SD    54671.035040
IN    53977.583226
OH    53796.919665
KS    53666.827834
AK    53612.781513
MI    52820.443205
OR    52714.642686
FL    52637.898271
ME    52089.824074
AZ    50057.323457
MT    49236.637119
ID    48970.826715
MO    48676.014677
NC    48204.376238
GA    47684.137415
OK    47369.271605
TN    45826.246815
LA    45205.025243
SC    44605.849057
AL    43627.909657
WV    42261.288952
KY    42046.053455
NM    41493.665761
AR    40887.957699
MS    38095.522459
PR    20483.648855
Nam

### Assemble quakes / tornadoes data

Initially collected affected zips manually via internet research. This task would have been near impossible for some of the hurricanes, at which point we began using get_affected_area() to estimate the affected area

In [28]:
ak_quake_zips = [
    99556,
    99568,
    99610,
    99572,
    99603,
    99605,
    99669,
    99611,
    99631,
    99639,
    99663,
    99664,
    99672,
    99682,
    99501,
    99502,
    99503,
    99504,
    99507,
    99508,
    99509,
    99510,
    99511,
    99513,
    99514,
    99515,
    99516,
    99517,
    99518,
    99519,
    99520,
    99521,
    99522,
    99523,
    99524,
    99529,
    99530,
    99599,
    99695,
    99645,
    99652,
    99654,
    99667,
    99674,
    99676,
    99683,
    99687,
    99688,
    99694,
    99574
]


In [29]:
kilauea_volc_zips = [
    96778,
    96754,
    96785,
    96722,
]

In [30]:
new_orleans_tornado_zips = [
    70112,
    70113,
    70114,
    70115,
    70116,
    70117,
    70118,
    70119,
    70121,
    70122,
    70123,
    70124,
    70125,
    70126,
    70127,
    70128,
    70129,
    70130,
    70131,
    70139,
    70145,
    70146,
    70163,
    70170
]

In [31]:
wisco_flood_zips = [
    54612,
    54610,
    54622,
    54629,
    54755,
    54756,
    54626,
    54628,
    54631,
    54645,
    53821,
    54654,
    54655,
    54657,
    53826,
    53825,
    53827,
    53518,
    53543,
    53554,
    53802,
    53801,
    53569,
    53804,
    53806,
    53573,
    53805,
    53808,
    53807,
    53810,
    53809,
    53812,
    53811,
    53816,
    53813,
    53818,
    53817,
    53821,
    53820,
    53502,
    53521,
]

In [32]:
ca_quake_zips = [
    93215,
    93220,
    93222,
    93461,
    93224,
    93226,
    93225,
    93502,
    93501,
    93505,
    93504,
    93238,
    93241,
    93240,
    93518,
    93519,
    93524,
    93523,
    93527,
    93250,
    93249,
    93252,
    93251,
    93531,
    93255,
    93263,
    93268,
    93276,
    93554,
    93556,
    93561,
    93560,
    93280,
    93285,
    93283,
    93287,
    93581,
    93301,
    93303,
    93305,
    93304,
    93307,
    93306,
    93309,
    93308,
    93312,
    93311,
    93314,
    93313,
    93380,
    93383,
    93387,
    93389,
    93388,
    93390,
    93203,
    93205,
    93206
]

In [33]:
ga_tornado_zips = [
    31620,
    31627,
    31637,
    31647,
    31625,
    31629,
    31638,
    31643,
    31622,
    31639,
    31645,
    31720,
    31749,
    31701,
    31702,
    31703,
    31704,
    31705,
    31706,
    31707,
    31708,
    31721,
    31782
]

In [34]:
wa_tornado_zips = [
    98322,
    98392,
    98337,
    98340,
    98342,
    98345,
    98346,
    98353,
    98359,
    98364,
    98366,
    98367,
    98370,
    98110,
    98380,
    98384,
    98310,
    98383,
    98312,
    98311,
    98315,
    98314
]

In [35]:
# No longer using this FEMA name to link data.
ak_quake_name = 'USA: Earthquake - Anchorage (Alaska) - 30/11/2018'
kilauea_volc_name = 'United States: Volcano eruption - Mount Kilauea; Hawaii - 04/05/2018'
new_orleans_tornado_name = 'United States : Tornado - Louisiana - 07/02/2017'
wisco_flood_name = 'United States of America: Arcadia Flooding - Wisconsin - 20/07/2017'
ca_quake_name = 'California Earthquakes (EM-3415)'
ga_tornado_name = 'United States:  Southern Georgia Tornado - 25/01/2017'
wa_tornado_name =  'USA: Tornado - Kitsap County (Washington) - 18/12/2018'

In [36]:
# Severity will be converted to 1-5 scale
ak_quake = ['Anchorage Earthquake', 'earthquake', 3.5, ak_quake_zips, 317, 7.6e7]
kilauea_volc = ['Kilauea Quake + Volcano', 'earthquake', 3.4, kilauea_volc_zips, 2800, 8e8]
new_orleans_tornado = ['New Orleans Tornado', 'tornado', 3, new_orleans_tornado_zips, 419, 2.7e6]
wisco_flood = ['Wisconsin Flood', 'flood', 2, wisco_flood_zips, 200, 1e7]
ca_quake = ['California Earthquake', 'earthquake', 3.5, ca_quake_zips, 100, 2e8]
ga_tornado = ['Georgia Tornado', 'tornado', 3, ga_tornado_zips, 2860, 1.3e9]
wa_tornado = ['Washington Tornado', 'tornado', 2, wa_tornado_zips, 125, 1.8e6]

ind_diz = [
    ak_quake,
    kilauea_volc,
    new_orleans_tornado,
    wisco_flood,
    ca_quake,
    ga_tornado,
    wa_tornado
]

In [37]:
ind_dis = pd.DataFrame(columns = ['event', 
                                  'type', 
                                  'severity', 
                                  'affected_zips', 
                                  'displaced',
                                  'damage',
                                 ])

In [38]:
for i, diz in enumerate(ind_diz):
    ind_dis.loc[i] = diz

In [39]:
ind_dis

,event,type,severity,affected_zips,displaced,damage
0,Anchorage Earthquake,earthquake,3.5,"[99556, 99568, 99610, 99572, 99603, 99605, 996...",317,7.600000e+07
1,Kilauea Quake + Volcano,earthquake,3.4,"[96778, 96754, 96785, 96722]",2800,8.000000e+08
2,New Orleans Tornado,tornado,3.0,"[70112, 70113, 70114, 70115, 70116, 70117, 701...",419,2.700000e+06
3,Wisconsin Flood,flood,2.0,"[54612, 54610, 54622, 54629, 54755, 54756, 546...",200,1.000000e+07
4,California Earthquake,earthquake,3.5,"[93215, 93220, 93222, 93461, 93224, 93226, 932...",100,2.000000e+08
5,Georgia Tornado,tornado,3.0,"[31620, 31627, 31637, 31647, 31625, 31629, 316...",2860,1.300000e+09
6,Washington Tornado,tornado,2.0,"[98322, 98392, 98337, 98340, 98342, 98345, 983...",125,1.800000e+06


### Clean Hurricanes

In [40]:
hurricanes = pd.read_csv('./data/hurricanes.csv')

In [41]:
hurricanes

,Unnamed: 0,event,date,category,area_affected,damage,fatalities,displaced
0,1,HURRICANE BRET,"August 18, 1999 – August 25, 1999",Category 4 Hurricane (SSHWS),Mexico and Texas,$15 million (1999 USD),7,180000
1,6,HURRICANE IRMA,"August 30, 2017 – September 14, 2017","Category 5 Hurricane (SSHWS), Category 1 Hurri...","Florida, Saint Martin, Puerto Rico, and more",NaN,134,201689
2,7,HURRICANE ANDREW,"August 16, 1992 – August 28, 1992",Category 5 Hurricane (SSHWS),"The Bahamas, Florida, Southeastern United Stat...",$27.3 billion (1992 USD),65,250000
3,20,HURRICANE LANE,"August 15, 2018 – August 29, 2018",Category 5 Hurricane (SSHWS),NaN,≥ $42.5 million (2018 USD),1,1753
4,31,HURRICANE IKE,"September 1, 2008 – September 15, 2008",Category 4 Hurricane (SSHWS),"Canada, Cuba, Dominican Republic, and more",$38 billion (2008 USD); (Sixth-costliest hurri...,195,35200
5,48,HURRICANE MARIA,"September 16, 2017 – October 2, 2017",Category 5 Hurricane (SSHWS),"Puerto Rico, Dominica, Dominican Republic, and...",NaN,"3,057",130000
6,55,HURRICANE MICHAEL,"October 7, 2018 – October 16, 2018",Category 5 Hurricane (SSHWS),"Florida, Cuba, Mexico Beach, and more",$25.1 billion (2018 USD),NaN,375000
7,68,HURRICANE SANDY,"October 22, 2012 – November 2, 2012",Category 3 Hurricane (SSHWS),"Puerto Rico, Jamaica, Haiti, and more",NaN,285,25000
8,73,HURRICANE FLORENCE,"August 31, 2018 – September 18, 2018",Category 4 Hurricane (SSHWS),"Cape Verde, North Carolina, Bermuda, and more",$24.23 billion (2018 USD),53,463674


In [107]:
hurricanes.shape

(9, 6)

#### Clean category

In [42]:
def map_hurricane_cat(s):
    return s.split()[1]

In [43]:
hurricanes['category'] = hurricanes['category'].apply(map_hurricane_cat).astype(float)

#### Clean damage cost

In [44]:
# Example: "$15 Million" should be 15000000
def map_hurricane_damage(s):
    if isinstance(s, float):
        return s
    else:
        for word in s.split():
            if word[0] == '$':
                n = float(word[1:])
        for word in s.split():
            if word == 'million':
                return n * 1_000_000
            elif word == 'billion':
                return n * 1_000_000_000

In [45]:
hurricanes['damage'] = hurricanes['damage'].apply(map_hurricane_damage)

In [46]:
hurricanes

,Unnamed: 0,event,date,category,area_affected,damage,fatalities,displaced
0,1,HURRICANE BRET,"August 18, 1999 – August 25, 1999",4.0,Mexico and Texas,1.500000e+07,7,180000
1,6,HURRICANE IRMA,"August 30, 2017 – September 14, 2017",5.0,"Florida, Saint Martin, Puerto Rico, and more",NaN,134,201689
2,7,HURRICANE ANDREW,"August 16, 1992 – August 28, 1992",5.0,"The Bahamas, Florida, Southeastern United Stat...",2.730000e+10,65,250000
3,20,HURRICANE LANE,"August 15, 2018 – August 29, 2018",5.0,NaN,4.250000e+07,1,1753
4,31,HURRICANE IKE,"September 1, 2008 – September 15, 2008",4.0,"Canada, Cuba, Dominican Republic, and more",3.800000e+10,195,35200
5,48,HURRICANE MARIA,"September 16, 2017 – October 2, 2017",5.0,"Puerto Rico, Dominica, Dominican Republic, and...",NaN,"3,057",130000
6,55,HURRICANE MICHAEL,"October 7, 2018 – October 16, 2018",5.0,"Florida, Cuba, Mexico Beach, and more",2.510000e+10,NaN,375000
7,68,HURRICANE SANDY,"October 22, 2012 – November 2, 2012",3.0,"Puerto Rico, Jamaica, Haiti, and more",NaN,285,25000
8,73,HURRICANE FLORENCE,"August 31, 2018 – September 18, 2018",4.0,"Cape Verde, North Carolina, Bermuda, and more",2.423000e+10,53,463674


In [47]:
# missing values for damage
hurricanes.loc[1, 'damage'] = 5e10
hurricanes.loc[5, 'damage'] = 9.5e10
hurricanes.loc[7, 'damage'] = 6.5e10

In [48]:
# Rename
hurricanes.rename(columns={'category': 'severity'}, inplace = True)

#### Get affected zips

In [49]:
def get_affected_area(central_zip, severity):
    
    # Center is a tuple of coordinates for central zip
    center = (float(df[df['ZCTA'] == central_zip]['lat']), float(df[df['ZCTA'] == central_zip]['lng']))
    
    # Each point in severity equates to a degree latitude / longitude
    polygon = Polygon([
        (center[0] - severity, center[1]), 
        (center[0], center[1] + severity),
        (center[0] + severity, center[1]),
        (center[0], center[1] - severity)
    ])

    # return list of all zips within the effected area
    affected_zips = []
    for i in range(df.shape[0]):
        point = Point(df.loc[i, 'lat'], df.loc[i, 'lng'])
        if polygon.contains(point):
            affected_zips.append(df.loc[i, 'ZCTA'])
    return affected_zips

In [50]:
hurricanes['central_zip'] = 0
hurricanes.loc[hurricanes['event'] == 'HURRICANE BRET', 'central_zip'] = 78374
hurricanes.loc[hurricanes['event'] == 'HURRICANE IRMA', 'central_zip'] = 33001
hurricanes.loc[hurricanes['event'] == 'HURRICANE ANDREW', 'central_zip'] = 34974
hurricanes.loc[hurricanes['event'] == 'HURRICANE LANE', 'central_zip'] = 96720
hurricanes.loc[hurricanes['event'] == 'HURRICANE IKE', 'central_zip'] = 77550
hurricanes.loc[hurricanes['event'] == 'HURRICANE MARIA', 'central_zip'] = 601
hurricanes.loc[hurricanes['event'] == 'HURRICANE MICHAEL', 'central_zip'] = 32401
hurricanes.loc[hurricanes['event'] == 'HURRICANE SANDY', 'central_zip'] = 20012
hurricanes.loc[hurricanes['event'] == 'HURRICANE FLORENCE', 'central_zip'] = 28480

In [51]:
hurricanes['affected_zips'] = np.array

In [52]:
for idx in hurricanes.index:
    hurricanes.at[idx, 'affected_zips'] = get_affected_area(
        hurricanes.loc[idx, 'central_zip'], hurricanes.loc[idx, 'severity'])

#### Merge hurricanes with quakes / tornadoes

In [53]:
hurricanes['type'] = 'hurricane'

In [54]:
hurricanes.drop(columns = ['Unnamed: 0', 'date', 'area_affected', 'fatalities', 'central_zip'], inplace = True)

In [55]:
hurricanes = hurricanes[['event', 'type', 'severity', 'affected_zips', 'displaced', 'damage']]

In [56]:
ind_dis = pd.concat([ind_dis, hurricanes], ignore_index = True)

In [57]:
ind_dis

,event,type,severity,affected_zips,displaced,damage
0,Anchorage Earthquake,earthquake,3.5,"[99556, 99568, 99610, 99572, 99603, 99605, 996...",317,7.600000e+07
1,Kilauea Quake + Volcano,earthquake,3.4,"[96778, 96754, 96785, 96722]",2800,8.000000e+08
2,New Orleans Tornado,tornado,3.0,"[70112, 70113, 70114, 70115, 70116, 70117, 701...",419,2.700000e+06
3,Wisconsin Flood,flood,2.0,"[54612, 54610, 54622, 54629, 54755, 54756, 546...",200,1.000000e+07
4,California Earthquake,earthquake,3.5,"[93215, 93220, 93222, 93461, 93224, 93226, 932...",100,2.000000e+08
5,Georgia Tornado,tornado,3.0,"[31620, 31627, 31637, 31647, 31625, 31629, 316...",2860,1.300000e+09
6,Washington Tornado,tornado,2.0,"[98322, 98392, 98337, 98340, 98342, 98345, 983...",125,1.800000e+06
7,HURRICANE BRET,hurricane,4.0,"[76501, 76502, 76504, 76508, 76511, 76513, 765...",180000,1.500000e+07
8,HURRICANE IRMA,hurricane,5.0,"[32114, 32117, 32118, 32119, 32124, 32127, 321...",201689,5.000000e+10
9,HURRICANE ANDREW,hurricane,5.0,"[31304, 31305, 31327, 31328, 31331, 31406, 314...",250000,2.730000e+10


### Clean wildfires

In [58]:
fires = pd.read_csv('./data/wildfires.csv')

In [59]:
fires

,Unnamed: 0,df_id,name,type,days without containment,Affected Zips,Damage,displaced people
0,NaN,6076.0,camp fire,wildfre,NaN,"95914, 95916, 95917, 95973, 95928, 95926, 9593...",8.400000e+09,53400.0
1,NaN,5954.0,old fire,wildfre,NaN,"92404, 92346, 92407, 92410, 92405, 92325, 92322.",4.803000e+08,5000.0
2,2013.0,5822.0,Colorado wildfires (Black Forest fire),wildfre,20.0,"80106, 80132, 80133, 80808, 80809, 80817, 8081...",9.000000e+07,43416.0
3,2016.0,5939.0,blue cut fire,wildfre,7.0,"92345, 92397, 92358, 93545, 92407, 92392, 9201...",NaN,82000.0
4,2017.0,6014.0,Caribou Fire - Montana,wildfre,83.0,59930,NaN,100.0
5,NaN,NaN,NaN,wildfre,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,wildfre,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
fires.drop(fires.index[4:], inplace = True) #only first 4 are usable

In [61]:
# Assign severity manually based on damage
fires['severity'] = 0
fires.loc[0, 'severity'] = 4
fires.loc[1, 'severity'] = 3
fires.loc[2, 'severity'] = 3
fires.loc[3, 'severity'] = 3
fires.loc[3, 'Damage'] = 18_000_000

In [62]:
fires['type'] = 'wildfire'

In [63]:
fires.loc[1, 'Affected Zips'] = fires.loc[1, 'Affected Zips'].replace('.','')

In [64]:
# needs to be a list but came in as a str
def clean_fire_zips(s):
    ls = s.replace(',','').split()
    ls1 = []
    for s in ls:
        ls1.append(int(s))
    return ls1

In [65]:
fires['affected_zips'] = np.array

In [66]:
fires['affected_zips'] = fires['Affected Zips'].apply(clean_fire_zips)

#### Merge wildfires

In [67]:
fires.rename(columns = {
    'name' : 'event',
    'Damage' : 'damage',
    'displaced people' : 'displaced'}, inplace = True)

In [68]:
fires = fires[['event', 'type', 'severity', 'affected_zips', 'displaced', 'damage']]

In [69]:
ind_dis = pd.concat([ind_dis, fires], ignore_index = True)

In [70]:
ind_dis['affected_zips_count'] = ind_dis['affected_zips'].apply(lambda x: len(x))

In [71]:
ind_dis

,event,type,severity,affected_zips,displaced,damage,affected_zips_count
0,Anchorage Earthquake,earthquake,3.5,"[99556, 99568, 99610, 99572, 99603, 99605, 996...",317,7.600000e+07,50
1,Kilauea Quake + Volcano,earthquake,3.4,"[96778, 96754, 96785, 96722]",2800,8.000000e+08,4
2,New Orleans Tornado,tornado,3.0,"[70112, 70113, 70114, 70115, 70116, 70117, 701...",419,2.700000e+06,24
3,Wisconsin Flood,flood,2.0,"[54612, 54610, 54622, 54629, 54755, 54756, 546...",200,1.000000e+07,41
4,California Earthquake,earthquake,3.5,"[93215, 93220, 93222, 93461, 93224, 93226, 932...",100,2.000000e+08,58
5,Georgia Tornado,tornado,3.0,"[31620, 31627, 31637, 31647, 31625, 31629, 316...",2860,1.300000e+09,23
6,Washington Tornado,tornado,2.0,"[98322, 98392, 98337, 98340, 98342, 98345, 983...",125,1.800000e+06,22
7,HURRICANE BRET,hurricane,4.0,"[76501, 76502, 76504, 76508, 76511, 76513, 765...",180000,1.500000e+07,724
8,HURRICANE IRMA,hurricane,5.0,"[32114, 32117, 32118, 32119, 32124, 32127, 321...",201689,5.000000e+10,613
9,HURRICANE ANDREW,hurricane,5.0,"[31304, 31305, 31327, 31328, 31331, 31406, 314...",250000,2.730000e+10,878


### Merge income data with business data

In [72]:
biz = pd.read_csv('./data/census_biz.csv')
biz.head()

,Unnamed: 0,EMP,EMPSZES,GEO_ID,PAYANN,ESTAB,YEAR,zip code
0,1,9891,1,8610000US01001,422470,473,2017,1001
1,8,8482,1,8610000US01002,299103,545,2017,1002
2,15,337,1,8610000US01003,10380,21,2017,1003
3,18,213,1,8610000US01004,5912,12,2017,1004
4,20,1248,1,8610000US01005,35287,94,2017,1005


In [73]:
df = pd.merge(df, biz, left_on = 'ZCTA', right_on = 'zip code')
df.head()

,ZCTA,n_households,med_income,population,density,city,state_id,state_name,lat,lng,Unnamed: 0,EMP,EMPSZES,GEO_ID,PAYANN,ESTAB,YEAR,zip code
0,1001,7460,57694,17621,562.8,Agawam,MA,Massachusetts,42.06258,-72.62590,1,9891,1,8610000US01001,422470,473,2017,1001
1,1002,9976,52379,30066,203.7,Amherst,MA,Massachusetts,42.37492,-72.46210,8,8482,1,8610000US01002,299103,545,2017,1002
2,1003,31,2500,11238,5629.7,Amherst,MA,Massachusetts,42.39192,-72.52479,15,337,1,8610000US01003,10380,21,2017,1003
3,1005,1785,70325,4991,44.3,Barre,MA,Massachusetts,42.42018,-72.10615,20,1248,1,8610000US01005,35287,94,2017,1005
4,1007,5558,86165,14967,107.5,Belchertown,MA,Massachusetts,42.27875,-72.40036,26,1939,1,8610000US01007,60005,223,2017,1007


In [74]:
df.shape

(29722, 18)

#### These functions get the area stats given the list of zips

In [75]:
def get_total_emps(zip_list):
    emps = 0
    for z in zip_list:
        try:
            emps += df[df['ZCTA'] == z]['EMP'].iloc[0]
        except:
            pass
    return emps

In [76]:
def get_avg_density(zip_list):
    total_dens = 0
    for z in zip_list:
        try:
            total_dens += df[df['ZCTA'] == z]['density'].iloc[0]
        except:
            pass
    avg_dens = total_dens / len(zip_list)
    return avg_dens

In [77]:
def get_households(zip_list):
    houses = 0
    for z in zip_list:
        try:
            houses += df[df['ZCTA'] == z]['n_households'].iloc[0]
        except:
            pass
    return houses

In [78]:
def get_pop_data(zip_list):
    pop = 0
    for z in zip_list:
        try:
            pop += df[df['ZCTA'] == z]['population'].iloc[0]
        except:
            pass
    return pop

In [79]:
ind_dis['total_population'] = ind_dis['affected_zips'].apply(get_pop_data)

In [80]:
def get_inc(zip_list):
    inc = 0
    for z in zip_list:
        try:
            inc += df[df['ZCTA'] == z]['med_income'].iloc[0]
        except:
            pass
    return inc

In [81]:
ind_dis['total_med_income'] = ind_dis['affected_zips'].apply(get_inc)

In [82]:
def get_payann(zip_list):
    payann = 0
    for z in zip_list:
        try:
            payann += df[df['ZCTA'] == z]['PAYANN'].iloc[0]
        except:
            pass
    return payann

In [83]:
ind_dis['total_PAYANN'] = ind_dis['affected_zips'].apply(get_payann)

In [84]:
def get_estab(zip_list):
    estab = 0
    for z in zip_list:
        try:
            estab += df[df['ZCTA'] == z]['ESTAB'].iloc[0]
        except:
            pass
    return estab

In [85]:
ind_dis['total_ESTAB'] = ind_dis['affected_zips'].apply(get_estab)

In [86]:
ind_dis

,event,type,severity,affected_zips,displaced,damage,affected_zips_count,total_population,total_med_income,total_PAYANN,total_ESTAB
0,Anchorage Earthquake,earthquake,3.5,"[99556, 99568, 99610, 99572, 99603, 99605, 996...",317,7.600000e+07,50,408398,2436379,9871745,12268
1,Kilauea Quake + Volcano,earthquake,3.4,"[96778, 96754, 96785, 96722]",2800,8.000000e+08,4,23772,220559,150118,372
2,New Orleans Tornado,tornado,3.0,"[70112, 70113, 70114, 70115, 70116, 70117, 701...",419,2.700000e+06,24,427938,899921,10146427,10969
3,Wisconsin Flood,flood,2.0,"[54612, 54610, 54622, 54629, 54755, 54756, 546...",200,1.000000e+07,41,107022,1924147,1302086,2437
4,California Earthquake,earthquake,3.5,"[93215, 93220, 93222, 93461, 93224, 93226, 932...",100,2.000000e+08,58,847018,2032645,8020244,12275
5,Georgia Tornado,tornado,3.0,"[31620, 31627, 31637, 31647, 31625, 31629, 316...",2860,1.300000e+09,23,147194,578867,1422120,2930
6,Washington Tornado,tornado,2.0,"[98322, 98392, 98337, 98340, 98342, 98345, 983...",125,1.800000e+06,22,262475,1487742,2418143,5893
7,HURRICANE BRET,hurricane,4.0,"[76501, 76502, 76504, 76508, 76511, 76513, 765...",180000,1.500000e+07,724,13298407,39534134,240318625,276051
8,HURRICANE IRMA,hurricane,5.0,"[32114, 32117, 32118, 32119, 32124, 32127, 321...",201689,5.000000e+10,613,15093629,33777125,245025313,425359
9,HURRICANE ANDREW,hurricane,5.0,"[31304, 31305, 31327, 31328, 31331, 31406, 314...",250000,2.730000e+10,878,19268584,46170588,297235735,522278


#### Add dis types

In [87]:
ind_dis = pd.merge(ind_dis, pd.get_dummies(ind_dis['type']), 
         left_on = ind_dis.index, right_on = pd.get_dummies(ind_dis['type']).index)

In [88]:
ind_dis.drop(columns = 'flood', inplace = True) # only one observation. acts as dropfirst

In [89]:
ind_dis

,key_0,event,type,severity,affected_zips,displaced,damage,affected_zips_count,total_population,total_med_income,total_PAYANN,total_ESTAB,earthquake,hurricane,tornado,wildfire
0,0,Anchorage Earthquake,earthquake,3.5,"[99556, 99568, 99610, 99572, 99603, 99605, 996...",317,7.600000e+07,50,408398,2436379,9871745,12268,1,0,0,0
1,1,Kilauea Quake + Volcano,earthquake,3.4,"[96778, 96754, 96785, 96722]",2800,8.000000e+08,4,23772,220559,150118,372,1,0,0,0
2,2,New Orleans Tornado,tornado,3.0,"[70112, 70113, 70114, 70115, 70116, 70117, 701...",419,2.700000e+06,24,427938,899921,10146427,10969,0,0,1,0
3,3,Wisconsin Flood,flood,2.0,"[54612, 54610, 54622, 54629, 54755, 54756, 546...",200,1.000000e+07,41,107022,1924147,1302086,2437,0,0,0,0
4,4,California Earthquake,earthquake,3.5,"[93215, 93220, 93222, 93461, 93224, 93226, 932...",100,2.000000e+08,58,847018,2032645,8020244,12275,1,0,0,0
5,5,Georgia Tornado,tornado,3.0,"[31620, 31627, 31637, 31647, 31625, 31629, 316...",2860,1.300000e+09,23,147194,578867,1422120,2930,0,0,1,0
6,6,Washington Tornado,tornado,2.0,"[98322, 98392, 98337, 98340, 98342, 98345, 983...",125,1.800000e+06,22,262475,1487742,2418143,5893,0,0,1,0
7,7,HURRICANE BRET,hurricane,4.0,"[76501, 76502, 76504, 76508, 76511, 76513, 765...",180000,1.500000e+07,724,13298407,39534134,240318625,276051,0,1,0,0
8,8,HURRICANE IRMA,hurricane,5.0,"[32114, 32117, 32118, 32119, 32124, 32127, 321...",201689,5.000000e+10,613,15093629,33777125,245025313,425359,0,1,0,0
9,9,HURRICANE ANDREW,hurricane,5.0,"[31304, 31305, 31327, 31328, 31331, 31406, 314...",250000,2.730000e+10,878,19268584,46170588,297235735,522278,0,1,0,0


### Modeling?

In [90]:
X = ind_dis[[
    'severity', 
    'affected_zips_count', 
    'total_population',
    'earthquake',
    'hurricane',
    'wildfire',
    'tornado'
]]
y = ind_dis['displaced']

In [91]:
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

In [92]:
# Used TTS to test out scoring but will train model on whole data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 42)

In [93]:
rf = RandomForestRegressor()
rf.fit(X_scaled, y)
#rf.fit(X_train, y_train)
#rf.score(X_train, y_train), rf.score(X_test, y_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [94]:
# pickle model
pickle.dump(rf, open("wageloss_rfr.pkl", "wb"))

### Input function

In [95]:
# get summary stats for whole affected area
def input_disaster(central_zip, severity):
    area_stats = {}
    affected_zips = get_affected_area(central_zip, severity)
    area_stats['total_population'] = get_pop_data(affected_zips)
    area_stats['total_households'] = get_households(affected_zips)
    area_stats['average_population_density'] = get_avg_density(affected_zips)
    area_stats['total_employees'] = get_total_emps(affected_zips)
    
    #area_stats['total_income'] = get_inc(affected_zips) * area_stats['total_households']
    area_stats['total_income'] = get_payann(affected_zips) * 1000 #payann is in thousands
    
    area_stats['total_businesses'] = get_estab(affected_zips)
    return area_stats

In [96]:
# testing
central_zip = 2110
severity = 3

In [97]:
input_disaster(central_zip, severity)

{'total_population': 12885184,
 'total_households': 4973336,
 'average_population_density': 617.172008701958,
 'total_employees': 5595572,
 'total_income': 328588709000,
 'total_businesses': 339466}

In [98]:
# predict with the model
def predict_displaced(central_zip, severity, dis_type):
    affected_zips_count = len(get_affected_area(central_zip, severity))
    total_pop = input_disaster(central_zip, severity)['total_population']
    
    earthquake = 0
    hurricane = 0
    tornado = 0
    wildfire = 0

    if dis_type == 'earthquake':
        earthquake = 1
    elif dis_type == 'hurricane':
        hurricane = 1
    elif dis_type == 'tornado':
        tornado = 1
    elif dis_type == 'wildfire':
        wildfire = 1
    
#     X_pred = ss.transform([[
#         severity, 
#         affected_zip_count, 
#         total_pop,
#         earthquake,
#         hurricane,
#         tornado,
#         wildfire
#     ]])

    data = [{
            'severity': severity,
            'affected_zips_count': affected_zips_count,
            'total_pop': total_pop,
            'earthquake': earthquake,
            'hurricane': hurricane,
            'tornado': tornado,
            'wildfire': wildfire,
            }]
    
    data_df = pd.DataFrame(data,index = [0])
    data_df = data_df[[
        'severity',
        'affected_zips_count',
        'total_pop',
        'earthquake',
        'hurricane',
        'tornado',
        'wildfire']]
        
    X_pred = ss.transform(data_df)
    #X_pred = StandardScaler().fit_transform(data_df)
    
    return rf.predict(X_pred)

In [99]:
predict_displaced(central_zip, severity, 'earthquake')

array([157345.4])

In [100]:
# def predict_loss(central_zip, severity, dis_type):
#     loss = {}
    
#     stats = input_disaster(central_zip, severity)
#     displaced = predict_displaced(central_zip, severity, dis_type)
#     percent_displaced = displaced / stats['total_population']
    
#     total_annual_loss = percent_displaced * stats['total_income']
#     total_daily_loss = total_annual_loss / 365

#     loss['Percent Displaced'] = percent_displaced
#     loss['Total Loss'] = total_daily_loss
#     return loss

In [101]:
# using model prediction and area stats to estimate total loss
def predict_loss(central_zip, severity, dis_type):
    loss = {}
    
    stats = input_disaster(central_zip, severity)
    
    earners_per_household = 1.2 # estimation
    earners = earners_per_household * stats['total_households'] # total earners in area
    percent_earners = earners / stats['total_population'] # proportion of population that are earners
    
    displaced = predict_displaced(central_zip, severity, dis_type) # our prediciton for displaced
    percent_displaced = displaced / stats['total_population']
    
    percent_displaced_earners = percent_displaced * percent_earners # only the earners impact the wage loss

    total_annual_loss = percent_displaced_earners * stats['total_income']
    total_daily_loss = total_annual_loss / 365

    loss['Percent Displaced'] = percent_displaced
    loss['Total Loss'] = total_daily_loss
    return loss

In [102]:
predict_loss(central_zip, severity, 'earthquake')

{'Percent Displaced': array([0.01221134]),
 'Total Loss': array([5091686.08507577])}

In [103]:
ind_dis.to_csv('./data/individual_disasters.csv')

In [104]:
df.to_csv('./data/census.csv')